### Baseline experiment

Experiment to compare the 4 Optimization algorithms before trying to improve sparse search

In [11]:
import sys  
sys.path.insert(0, '../')

import HPO
import pysgpp
import matplotlib.pyplot as plt
import matplotlib as mpl
import tensorflow as tf
import sklearn.metrics
from sklearn.model_selection import KFold
import numpy as np
import keras
from keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.compose import ColumnTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from numpy.random import seed
import random
import time
import os
import math
from matplotlib import cm
import matplotlib 
import matplotlib.patches as patches

import operator

from scipy.stats import norm

# matplotlib.use("pgf")
# matplotlib.rcParams.update({
#     "pgf.texsystem": "pdflatex",
#     'font.family': 'serif',
#     'text.usetex': True,
#     'pgf.rcfonts': False,
# })

random.seed(1)
seed(2)
tf.random.set_seed(3)

def reset_seeds():
    np.random.seed(1)
    random.seed(2)
    tf.random.set_seed(3)

VERBOSE = 0

SPARSE_RESULT = []
SPARSE_RESULT_OPTIMIZED = []

SPARSE_COST = []
SPARSE_COST_OPTIMIZED = []

### Hyperparameter space definition

In [12]:
hyperparameterspace = {
    'x0': ["interval", -2, 8],
    'x1': ["interval", -2, 8],
    'x2': ["interval", -2, 8],
    'x3': ["interval", -2, 8],
    'x4': ["interval", -2, 8],
    'x5': ["interval", -2, 8],
    'x6': ["interval", -2, 8],
    'x7': ["interval", -2, 8],
    'x8': ["interval", -2, 8],
    'x9': ["interval", -2, 8],
    #'learning_rate': ["interval-log", 0.000000001, 0.1],
    #'number_layers': ["interval-int", 1, 20],
    #'neurons_per_layer': ["interval-int", 1, 50]
}

hyperparameterspace_special = {}
for key in hyperparameterspace.keys():
    liste = []
    for i in range(1, len(hyperparameterspace[key])):
        liste.append(hyperparameterspace[key][i])
    hyperparameterspace_special[key] = liste



### Experiment parameters

In [13]:


BUDGETS = [1 + i * 52 for i in range(19)]

In [14]:
def visualize(points, adaptivity, alternative):
    x_values = []
    y_values = []
    z_values = []

    for i in range(len(points)):    
        x_values.append(points[i].get_coordinates()[0])
        y_values.append(points[i].get_coordinates()[1])
        z_values.append(points[i].get_value())


    fig = plt.figure()
    ax = plt.axes()
    surface = plt.scatter(x_values, y_values, c=z_values, cmap="plasma")
    plt.scatter(points[0].get_coordinates()[0], points[0].get_coordinates()[1], c="white", marker="x")


    ax.set_xlim([-5, 10])
    ax.set_ylim([-5, 10])

    plt.colorbar(surface)
    plt.gca().set_aspect("equal")

    # plt.savefig("./Alternative_"+str(alternative)+"_adapt_"+str(adaptivity)+".pgf",bbox_inches='tight' )

    plt.show()

### Optimization

In [15]:

################## MODEL AND FUNCTION DEFINITION ####################

def evaluate_model(epochs, batch_size, x2, x3, x4, x5, x6, x7, x8, x9):
    #return epochs * batch_size
    ################### ROSENBROCK ###############
    # sum = 0
    # sum += (1-epochs)**2 + 100 * (batch_size - epochs**2) ** 2
    # return sum

    # ################# RASTRIGIN #################
    # sum = 0
    # sum += epochs ** 2 - 10 * math.cos(2 * math.pi * epochs)
    # sum += batch_size ** 2 - 10 * math.cos(2 * math.pi * batch_size)
    
    # return len(hyperparameterspace) * 10 + sum

    # ################# RASTRIGIN #################
    sum = 0
    sum += epochs ** 2 - 10 * math.cos(2 * math.pi * epochs)
    sum += batch_size ** 2 - 10 * math.cos(2 * math.pi * batch_size)
    sum += x2 ** 2 - 10 * math.cos(2 * math.pi * x2)
    sum += x3 ** 2 - 10 * math.cos(2 * math.pi * x3)
    sum += x4 ** 2 - 10 * math.cos(2 * math.pi * x4)
    sum += x5 ** 2 - 10 * math.cos(2 * math.pi * x5)
    sum += x2 ** 2 - 10 * math.cos(2 * math.pi * x6)
    sum += x3 ** 2 - 10 * math.cos(2 * math.pi * x7)
    sum += x4 ** 2 - 10 * math.cos(2 * math.pi * x8)
    sum += x5 ** 2 - 10 * math.cos(2 * math.pi * x9)
    return len(hyperparameterspace) * 10 + sum

def function(coordinates):
    return evaluate_model(coordinates[0], coordinates[1], coordinates[2], coordinates[3], coordinates[4], coordinates[5], coordinates[6], coordinates[7], coordinates[8], coordinates[9])


BUDGET = 5000
INIT_POINTS = 5

ADAPTIVITIES = [0.1 * i for i in range(11)]


ALT_1 = "{"
ALT_2 = "{"
ALT_3 = "{"


for adaptivity in ADAPTIVITIES:

    reset_seeds()

    optimizer = HPO.IterativeRandomOptimization(HPO.Dataset([],[]), function, hyperparameterspace, BUDGET, 0, adaptivity, init_points=INIT_POINTS, alternative=0)
    points = optimizer.fit()

    points.sort(key=operator.attrgetter('value'))

    ALT_1 += "(" + str(adaptivity) + "," + str(points[0].get_value()) + ")"


    reset_seeds()

    optimizer = HPO.IterativeRandomOptimization(HPO.Dataset([],[]), function, hyperparameterspace, BUDGET, 0, adaptivity, init_points=INIT_POINTS, alternative=1)
    points = optimizer.fit()

    points.sort(key=operator.attrgetter('value'))
    

    ALT_2 += "(" + str(adaptivity) + "," + str(points[0].get_value()) + ")"



    optimizer = HPO.IterativeRandomOptimization(HPO.Dataset([],[]), function, hyperparameterspace, BUDGET, 0, adaptivity, init_points=INIT_POINTS, alternative=2)
    points = optimizer.fit()

    points.sort(key=operator.attrgetter('value'))


    ALT_3 += "(" + str(adaptivity) + "," + str(points[0].get_value()) + ")"


print(ALT_1+"}")
print(ALT_2+"}")
print(ALT_3+"}")



{(0.0,162.761025073682)(0.1,162.761025073682)(0.2,162.761025073682)(0.30000000000000004,165.65760013033167)(0.4,168.64616716760546)(0.5,165.2768663253831)(0.6000000000000001,168.1834839451662)(0.7000000000000001,165.35086318493714)(0.8,143.1580199831282)(0.9,146.8933255133596)(1.0,142.9938345967389)}
{(0.0,37.09356147694522)(0.1,37.09356147694522)(0.2,36.81602227605653)(0.30000000000000004,38.68156741616422)(0.4,27.282614603454675)(0.5,31.73913303273261)(0.6000000000000001,28.605152304899946)(0.7000000000000001,21.519348408206483)(0.8,33.04214109501011)(0.9,41.83500106274237)(1.0,60.94266017470839)}
{(0.0,31.371096705820833)(0.1,31.371096705820833)(0.2,31.371096705820833)(0.30000000000000004,38.922820229144364)(0.4,24.140387463402618)(0.5,18.51902820363044)(0.6000000000000001,40.51677991948232)(0.7000000000000001,30.425044718671558)(0.8,38.233466954045774)(0.9,38.69731668001502)(1.0,55.559856664944796)}
